# Library

In [1]:
import pymysql
import re, getpass
from datetime import datetime, timedelta
import pandas as pd

# 함수

In [2]:
def record_fuction(table, record_text, opteamname, start_time):
    # 전역변수 선언부
    conn = None
    cur = None
    # 메인 코드
    password = getpass.getpass('비밀번호를 입력하세요: ')
    conn = pymysql.connect(host = 'localhost', port=3306, user='root', password=password, db='futsaldata', charset='utf8')
    cur = conn.cursor() # 커서생성
    section_list = record_text.split('\n\n') # 한 줄 띄어쓰기 기준 분할(쿼터별 분할)
    # 경기 날짜 추출
    date_section = section_list[0]
    date_p = re.compile('[0-9]{1,2}/[0-9]{1,2}')
    date_m = date_p.search(date_section)
    year = datetime.today().year
    month = int(date_m.group().split('/')[0])
    day = int(date_m.group().split('/')[1])
    dttime = f'{year}-{month:0>2}-{day:0>2} {start_time}:00:00'
    game_start_time = datetime.strptime(dttime, '%Y-%m-%d %H:%M:%S')
    time_delta = timedelta(minutes=15) # 한 쿼터 시간(기본 15분)

    record_section_list = section_list[1:-1] # 쿼터별 기록만 추출

    id_list, datetime_list, quarter_list, goal_list, assist_list, opteamname_list = [], [], [], [], [], []
    p = re.compile('[0-9]:[0-9]') # 쿼터 총 골 기록 추출
    
    # 총 기록 줄 수 확인(이 값과 df의 id의 최댓값(id_list의 길이)이 동일해야함.)
    total_lines = 0
    for one in record_section_list:
        sections = one.splitlines() # 쿼터 기록 한줄씩 분리
        temp_datetime_list, temp_quarter_list, temp_goal_list, temp_assist_list, temp_opteamname_list = [], [], [], [], []
        quarter = sections[0].split('Q')[0] # 몇 쿼터인지 추출
        quarter_start_time = str( game_start_time + time_delta * ( int(quarter) - 1 ) ) # 쿼터 시작 시간 계산
        # 해당 쿼터 기록이 몇개인지 확인하기
        temp_m = p.search(sections[0])
        our_goal = int(temp_m.group().split(':')[0]) # 우리 골 기록
        op_goal = int(temp_m.group().split(':')[1]) # 상대 골 기록
        quarter_record_line_num = our_goal + op_goal
        total_lines += quarter_record_line_num # 총 기록 줄  수 기록
        # 쿼터 정보 기록
        temp_quarter_list = [quarter] * quarter_record_line_num
        # 쿼터 시작 시간 기록
        temp_datetime_list = [quarter_start_time] * quarter_record_line_num
        # 상대팀 이름 기록
        temp_opteamname_list = [opteamname] * quarter_record_line_num
        # print(sections)

        records = sections[1:] # 해당 쿼터의 스코어 기록 제외(골, 어시만 남김)
        if len(records) == 0: # 0:0인 경우
            # print('0:0인 조건문 실행.')
            temp_quarter_list = [quarter] # 쿼터 정보 기록
            # 골, 어시 'NULL'로 채우기
            temp_goal_list, temp_assist_list = ['NULL'], ['NULL']
            # 쿼터 시작 시간 기록
            temp_datetime_list = [quarter_start_time]
            # 상대팀 이름 기록
            temp_opteamname_list = [opteamname]
            total_lines += 1 # 0:0인 경우 골, 어시 빈 값으로 1줄만 채움
        else: # 0:0 이 아닌 경우
            # print('0:0이 아닌 경우 조건문 실행.')
            for record in records:
                goal_assist = record.split(' ')
                if len(goal_assist) == 2: # 골과 어시 존재
                    temp_goal_list.append(goal_assist[0])
                    temp_assist_list.append(goal_assist[1])
                else: # 골만 존재
                    temp_goal_list.append(goal_assist[0])
                    temp_assist_list.append('NULL')
        if op_goal != 0: # 상대 골이 존재
            # print('상대 골 존재 조건문 실행')
            temp_goal_list += ['상대'] * op_goal
            temp_assist_list += ['NULL'] * op_goal
        # 쿼터별로 제대로 기록됐는지 확인
        # print((f'{quarter}Q'), len(temp_datetime_list), len(temp_quarter_list), len(temp_goal_list), len(temp_assist_list), len(temp_opteamname_list))
        datetime_list += temp_datetime_list
        quarter_list += temp_quarter_list
        goal_list += temp_goal_list
        assist_list += temp_assist_list
        opteamname_list += temp_opteamname_list

    id_list = [str(x) for x in range(1, len(datetime_list) + 1)] # 기록 줄 수
    # print('total :', len(datetime_list), len(quarter_list), len(goal_list), len(assist_list), len(opteamname_list))

    sql = ''
    for id, game_datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
        if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
            sql = f"INSERT INTO {table} VALUES(" + id + ",'" + game_datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
            # print(sql)
        elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
            sql = f"INSERT INTO {table} VALUES(" + id + ",'" + game_datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
            # print(sql)
        else: # 골, 어시 둘다 존재하지 않는 경우
            sql = f"INSERT INTO {table} VALUES(" + id + ",'" + game_datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
            # print(sql)
        cur.execute(sql) # sql문 실행
        conn.commit() # 저장
    conn.close() # 종료
    
    df = pd.DataFrame({'id':id_list, 'game_datetime':datetime_list, 'quarter':quarter_list, 'goal':goal_list, 'assist':assist_list, 'opteamname':opteamname_list})
    return df, total_lines

## 정보 입력

In [7]:
# 데이터 입력할 테이블명
# 테이블 명 : test_records, records
table = 'records'

# 상대팀 이름 입력(문자형)
opteamname = '아일랜드fc'

# 경기 시작 시간(24시간제, 숫자형)
start_time =  21

# 기록 텍스트
record_text = '''9/6 금 용산 싼다 골도움
참: 핵 트 잡 뭉 범 뀨 쫑 삐 후 _9명

1Q) 2:1 승
잡 트
범 핵

2Q) 3:0 승
뭉 후
후 뭉
뭉

3Q) 0:0 무

4Q) 1:1 무
뀨 잡

5Q) 1:0 승
후 뭉

6Q) 1:1 무
삐 트

7Q) 0:0 무

8Q) 3:1 승
삐 범
삐 뀨
뀨 삐

4승 4무'''

## 결과 확인

In [8]:
df, total_lines = record_fuction(table, record_text, opteamname, start_time)

In [9]:
df

,id,game_datetime,quarter,goal,assist,opteamname
0,1,2024-09-06 21:00:00,1,잡,트,아일랜드fc
1,2,2024-09-06 21:00:00,1,범,핵,아일랜드fc
2,3,2024-09-06 21:00:00,1,상대,NULL,아일랜드fc
3,4,2024-09-06 21:15:00,2,뭉,후,아일랜드fc
4,5,2024-09-06 21:15:00,2,후,뭉,아일랜드fc
5,6,2024-09-06 21:15:00,2,뭉,NULL,아일랜드fc
6,7,2024-09-06 21:30:00,3,NULL,NULL,아일랜드fc
7,8,2024-09-06 21:45:00,4,뀨,잡,아일랜드fc
8,9,2024-09-06 21:45:00,4,상대,NULL,아일랜드fc
9,10,2024-09-06 22:00:00,5,후,뭉,아일랜드fc


In [10]:
total_lines

17

# 테스트

## DB 접속

In [14]:
# 전역변수 선언부
conn = None
cur = None

# 메인 코드
password = input('비밀번호를 입력하세요')
conn = pymysql.connect(host = 'localhost', port=3306, user='root', password=password, db='futsaldata', charset='utf8')
cur = conn.cursor() # 커서생성

In [5]:
# sql = 'CREATE TABLE IF NOT EXISTS userTable (id char(4), userName char(10), email char(15), birthYear int)' # 테스트 테이블 생성
# cur.execute(sql) # sql문 실행

# conn.commit() # 저장

# conn.close() # 종료

## test_records 테이블 생성

In [10]:
sql = "CREATE TABLE test_records (ID int(11) DEFAULT NULL, DATE datetime DEFAULT NULL, OPTEAMNAME varchar(100) DEFAULT NULL, QUARTER tinyint(4) DEFAULT NULL, GOAL varchar(3) DEFAULT NULL, ASSIST varchar(3) DEFAULT NULL)" # test_records 테이블 생성
cur.execute(sql) # sql문 실행

conn.commit() # 저장

## records 테이블에 데이터 입력

### 2024-08-23 경기
    8/23 금 용산 싼다 골도움
    참: 핵 삐 트 범 뀨 후 잡 뭉 _8명

    1Q) 3:2 승
    후 범
    핵 삐
    후 삐

    2Q) 2:2 무
    핵 뀨
    후 잡

    3Q) 1:0 승
    뭉 후

    4Q) 5:2 승
    삐 뭉
    삐 잡
    삐 트
    뭉 잡
    잡 삐

    5Q) 2:2 무
    핵 삐
    삐 핵

    6Q) 1:0 승
    뭉 후

    7Q) 1:3 패
    뭉 삐

    8Q) 5:0 승
    잡 핵
    잡 트
    잡 핵
    잡
    잡 뀨

    5승 2무 1패

#### 1쿼터
    1Q) 3:2 승
    후 범
    핵 삐
    후 삐

In [33]:
goal_list = ['후', '핵', '후', '상대', '상대']
assist_list = ['범', '삐', '삐', 'NULL', 'NULL']
datetime_list = ['2024-08-23 22:00:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['1'] * len(goal_list)
id_num = 1
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

6

In [30]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(1,'2024-08-23 22:00:00','fc intentar',1,'후','범')
INSERT INTO test_records VALUES(2,'2024-08-23 22:00:00','fc intentar',1,'핵','삐')
INSERT INTO test_records VALUES(3,'2024-08-23 22:00:00','fc intentar',1,'후','삐')
INSERT INTO test_records VALUES(4,'2024-08-23 22:00:00','fc intentar',1,'상대',NULL)
INSERT INTO test_records VALUES(5,'2024-08-23 22:00:00','fc intentar',1,'상대',NULL)


#### 2쿼터
    2Q) 2:2 무
    핵 뀨
    후 잡

In [34]:
goal_list = ['핵', '후', '상대', '상대']
assist_list = ['뀨', '잡', 'NULL', 'NULL']
datetime_list = ['2024-08-23 22:15:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['2'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

10

In [35]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(6,'2024-08-23 22:15:00','fc intentar',2,'핵','뀨')
INSERT INTO test_records VALUES(7,'2024-08-23 22:15:00','fc intentar',2,'후','잡')
INSERT INTO test_records VALUES(8,'2024-08-23 22:15:00','fc intentar',2,'상대',NULL)
INSERT INTO test_records VALUES(9,'2024-08-23 22:15:00','fc intentar',2,'상대',NULL)


#### 3쿼터
    3Q) 1:0 승
    뭉 후

In [36]:
goal_list = ['뭉']
assist_list = ['후']
datetime_list = ['2024-08-23 22:30:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['3'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

11

In [37]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(10,'2024-08-23 22:30:00','fc intentar',3,'뭉','후')


#### 4쿼터
    4Q) 5:2 승
    삐 뭉
    삐 잡
    삐 트
    뭉 잡
    잡 삐

In [38]:
goal_list = ['삐', '삐', '삐', '뭉', '잡', '상대', '상대']
assist_list = ['뭉', '잡', '트', '잡', '삐', 'NULL', 'NULL']
datetime_list = ['2024-08-23 22:45:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['4'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

18

In [39]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(11,'2024-08-23 22:45:00','fc intentar',4,'삐','뭉')
INSERT INTO test_records VALUES(12,'2024-08-23 22:45:00','fc intentar',4,'삐','잡')
INSERT INTO test_records VALUES(13,'2024-08-23 22:45:00','fc intentar',4,'삐','트')
INSERT INTO test_records VALUES(14,'2024-08-23 22:45:00','fc intentar',4,'뭉','잡')
INSERT INTO test_records VALUES(15,'2024-08-23 22:45:00','fc intentar',4,'잡','삐')
INSERT INTO test_records VALUES(16,'2024-08-23 22:45:00','fc intentar',4,'상대',NULL)
INSERT INTO test_records VALUES(17,'2024-08-23 22:45:00','fc intentar',4,'상대',NULL)


#### 5쿼터
    5Q) 2:2 무
    핵 삐
    삐 핵

In [40]:
goal_list = ['핵', '삐', '상대', '상대']
assist_list = ['삐', '핵', 'NULL', 'NULL']
datetime_list = ['2024-08-23 23:00:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['5'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

22

In [41]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(18,'2024-08-23 23:00:00','fc intentar',5,'핵','삐')
INSERT INTO test_records VALUES(19,'2024-08-23 23:00:00','fc intentar',5,'삐','핵')
INSERT INTO test_records VALUES(20,'2024-08-23 23:00:00','fc intentar',5,'상대',NULL)
INSERT INTO test_records VALUES(21,'2024-08-23 23:00:00','fc intentar',5,'상대',NULL)


#### 6쿼터
    6Q) 1:0 승
    뭉 후

In [42]:
goal_list = ['뭉']
assist_list = ['후']
datetime_list = ['2024-08-23 23:15:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['6'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

23

In [43]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(22,'2024-08-23 23:15:00','fc intentar',6,'뭉','후')


#### 7쿼터
    7Q) 1:3 패
    뭉 삐

In [44]:
goal_list = ['뭉', '상대', '상대', '상대']
assist_list = ['삐', 'NULL', 'NULL', 'NULL']
datetime_list = ['2024-08-23 23:30:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['7'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

27

In [45]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(23,'2024-08-23 23:30:00','fc intentar',7,'뭉','삐')
INSERT INTO test_records VALUES(24,'2024-08-23 23:30:00','fc intentar',7,'상대',NULL)
INSERT INTO test_records VALUES(25,'2024-08-23 23:30:00','fc intentar',7,'상대',NULL)
INSERT INTO test_records VALUES(26,'2024-08-23 23:30:00','fc intentar',7,'상대',NULL)


#### 8쿼터
    8Q) 5:0 승
    잡 핵
    잡 트
    잡 핵
    잡
    잡 뀨

In [46]:
goal_list = ['후', '핵', '후', '상대', '상대']
assist_list = ['범', '삐', '삐', 'NULL', 'NULL']
datetime_list = ['2024-08-23 23:45:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['8'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

32

In [47]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(27,'2024-08-23 23:45:00','fc intentar',8,'후','범')
INSERT INTO test_records VALUES(28,'2024-08-23 23:45:00','fc intentar',8,'핵','삐')
INSERT INTO test_records VALUES(29,'2024-08-23 23:45:00','fc intentar',8,'후','삐')
INSERT INTO test_records VALUES(30,'2024-08-23 23:45:00','fc intentar',8,'상대',NULL)
INSERT INTO test_records VALUES(31,'2024-08-23 23:45:00','fc intentar',8,'상대',NULL)


In [48]:
conn.commit() # 저장

conn.close() # 종료

### 2024-09-06 경기
    9/6 금 용산 싼다 골도움
    참: 핵 트 잡 뭉 범 뀨 쫑 삐 후 _9명

    1Q) 2:1 승
    잡 트
    범 핵

    2Q) 3:0 승
    뭉 후
    후 뭉
    뭉

    3Q) 0:0 무

    4Q) 1:1 무
    뀨 잡

    5Q) 1:0 승
    후 뭉

    6Q) 1:1 무
    삐 트

    7Q) 0:0 무

    8Q) 3:1 승
    삐 범
    삐 뀨
    뀨 삐

    4승 4무

#### 1쿼터
    1Q) 2:1 승
    잡 트
    범 핵

In [187]:
id_num = 1

In [188]:
goal_list = ['잡', '범', '상대']
assist_list = ['트', '핵', 'NULL']
datetime_list = ['2024-09-06 21:00:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['1'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

INSERT INTO records VALUES(1,'2024-09-06 21:00:00','아일랜드fc',1,'잡','트')
INSERT INTO records VALUES(2,'2024-09-06 21:00:00','아일랜드fc',1,'범','핵')
INSERT INTO records VALUES(3,'2024-09-06 21:00:00','아일랜드fc',1,'상대',NULL)


#### 2쿼터
    2Q) 3:0 승
    뭉 후
    후 뭉
    뭉

In [17]:
goal_list = ['뭉', '후', '뭉']
assist_list = ['후', '뭉', 'NULL']
datetime_list = ['2024-09-06 21:15:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['2'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

INSERT INTO records VALUES(4,'2024-09-06 21:15:00','아일랜드fc',2,'뭉','후')
INSERT INTO records VALUES(5,'2024-09-06 21:15:00','아일랜드fc',2,'후','뭉')
INSERT INTO records VALUES(6,'2024-09-06 21:15:00','아일랜드fc',2,'뭉',NULL)


#### 3쿼터
    3Q) 0:0 무

In [189]:
goal_list = ['NULL']
assist_list = ['NULL']
datetime_list = ['2024-09-06 21:30:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['3'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

INSERT INTO records VALUES(4,'2024-09-06 21:30:00','아일랜드fc',3,NULL,NULL)


#### 4쿼터
    4Q) 1:1 무
    뀨 잡

In [19]:
goal_list = ['뀨', '상대']
assist_list = ['잡', 'NULL']
datetime_list = ['2024-09-06 21:45:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['4'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

INSERT INTO records VALUES(8,'2024-09-06 21:45:00','아일랜드fc',4,'뀨','잡')
INSERT INTO records VALUES(9,'2024-09-06 21:45:00','아일랜드fc',4,'상대',NULL)


#### 5쿼터
    5Q) 1:0 승
    후 뭉

In [20]:
goal_list = ['후']
assist_list = ['뭉']
datetime_list = ['2024-09-06 22:00:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['5'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

INSERT INTO records VALUES(10,'2024-09-06 22:00:00','아일랜드fc',5,'후','뭉')


#### 6쿼터
    6Q) 1:1 무
    삐 트

In [21]:
goal_list = ['삐', '상대']
assist_list = ['트', 'NULL']
datetime_list = ['2024-09-06 22:15:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['6'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

INSERT INTO records VALUES(11,'2024-09-06 22:15:00','아일랜드fc',6,'삐','트')
INSERT INTO records VALUES(12,'2024-09-06 22:15:00','아일랜드fc',6,'상대',NULL)


#### 7쿼터
    7Q) 0:0 무

In [22]:
goal_list = ['NULL']
assist_list = ['NULL']
datetime_list = ['2024-09-06 22:30:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['7'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

INSERT INTO records VALUES(13,'2024-09-06 22:30:00','아일랜드fc',7,NULL,NULL)


#### 8쿼터
    8Q) 3:1 승
    삐 범
    삐 뀨
    뀨 삐

In [186]:
goal_list = ['삐', '삐', '뀨', '상대']
assist_list = ['범', '뀨', '삐', 'NULL']
datetime_list = ['2024-09-06 22:45:00'] * len(goal_list)
opteamname_list = ['아일랜드fc'] * len(goal_list)
quarter_list = ['8'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if goal != 'NULL' and assist != 'NULL': # 골, 어시 둘 다 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    elif goal != 'NULL' and assist == 'NULL': # 골만 존재하는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    else: # 골, 어시 둘다 존재하지 않는 경우
        sql = "INSERT INTO records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + "," + goal + "," + assist + ")"
        print(sql)
    # cur.execute(sql) # sql문 실행

NameError: name 'id_num' is not defined

In [24]:
conn.commit() # 저장

conn.close() # 종료

## 텍스트를 데이터 기록으로 변환
- 텍스트에서 필요한 정보
    - 경기 날짜
    - 쿼터
    - 골 선수명
    - 어시스트 선수명
    - 상대 골 수
- 그 외(텍스트 입력 활용)
    - 상대 팀명

In [113]:
# 샘플 텍스트
record_text = '''8/23 금 용산 싼다 골도움
참: 핵 삐 트 범 뀨 후 잡 뭉 _8명

1Q) 3:2 승
후 범
핵 삐
후 삐

2Q) 2:2 무
핵 뀨
후 잡

3Q) 1:0 승
뭉 후

4Q) 5:2 승
삐 뭉
삐 잡
삐 트
뭉 잡
잡 삐

5Q) 2:2 무
핵 삐
삐 핵

6Q) 1:0 승
뭉 후

7Q) 1:3 패
뭉 삐

8Q) 5:0 승
잡 핵
잡 트
잡 핵
잡
잡 뀨

5승 2무 1패'''

In [114]:
# 1. 전체 텍스트를 줄 바꿈 기준으로 분리
# 2. 첫번째 원소는 날짜 정보 얻기
# 3. 두번째 원소부터는 쿼터, 골, 어시스트, 상대 득점 정보 얻기

In [115]:
section_list = record_text.split('\n\n')
section_list

['8/23 금 용산 싼다 골도움\n참: 핵 삐 트 범 뀨 후 잡 뭉 _8명',
 '1Q) 3:2 승\n후 범\n핵 삐\n후 삐',
 '2Q) 2:2 무\n핵 뀨\n후 잡',
 '3Q) 1:0 승\n뭉 후',
 '4Q) 5:2 승\n삐 뭉\n삐 잡\n삐 트\n뭉 잡\n잡 삐',
 '5Q) 2:2 무\n핵 삐\n삐 핵',
 '6Q) 1:0 승\n뭉 후',
 '7Q) 1:3 패\n뭉 삐',
 '8Q) 5:0 승\n잡 핵\n잡 트\n잡 핵\n잡\n잡 뀨',
 '5승 2무 1패']

In [116]:
date_section = section_list[0]
date_section

'8/23 금 용산 싼다 골도움\n참: 핵 삐 트 범 뀨 후 잡 뭉 _8명'

In [117]:
date_p = re.compile('[0-9]{1,2}/[0-9]{1,2}')
date_m = date_p.search(date_section)
print(date_m.group())

8/23


In [118]:
year = dt.today().year
month = int(date_m.group().split('/')[0])
day = int(date_m.group().split('/')[1])
year, month, day

(2024, 8, 23)

In [119]:
current_date = f'{year}-{month:0>2}-{day:0>2} '
current_date

'2024-08-23 '

In [120]:
record_section_list = section_list[1:-1]
record_section_list

['1Q) 3:2 승\n후 범\n핵 삐\n후 삐',
 '2Q) 2:2 무\n핵 뀨\n후 잡',
 '3Q) 1:0 승\n뭉 후',
 '4Q) 5:2 승\n삐 뭉\n삐 잡\n삐 트\n뭉 잡\n잡 삐',
 '5Q) 2:2 무\n핵 삐\n삐 핵',
 '6Q) 1:0 승\n뭉 후',
 '7Q) 1:3 패\n뭉 삐',
 '8Q) 5:0 승\n잡 핵\n잡 트\n잡 핵\n잡\n잡 뀨']

In [121]:
sections = record_section_list[0].splitlines()
sections

['1Q) 3:2 승', '후 범', '핵 삐', '후 삐']

In [124]:
sections[0]

'1Q) 3:2 승'

In [125]:
sections[0].split('Q')[0]

'1'

In [126]:
p = re.compile('[0-9]:[0-9]')
m = p.search(sections[0])
print(m.group())
our_goal = int(m.group().split(':')[0])
op_goal = int(m.group().split(':')[1])
quarter_record_line_num = our_goal + op_goal
quarter_record_line_num

3:2


5

In [127]:
temp_quarter_list = []
temp = sections[0].split('Q')[0]
if quarter_record_line_num != 0:
    temp_quarter_list = [temp]*quarter_record_line_num
else:
    temp_quarter_list = [temp]
temp_quarter_list

['1', '1', '1', '1', '1']

In [128]:
sections[1:]

['후 범', '핵 삐', '후 삐']

In [129]:
sections[1]

'후 범'

In [130]:
sections[1].split(' ')

['후', '범']

In [131]:
sections[2].split(' ')

['핵', '삐']

In [133]:
temp_goal_list, temp_assist_list = [], []
print(temp_goal_list)
print(temp_assist_list)

[]
[]


In [134]:
if len(sections) != 1:
    for i, one in enumerate(sections[1:]):
        goal_assist = one.split(' ')
        if len(goal_assist) != 1:
            temp_goal_list.append(goal_assist[0])
            temp_assist_list.append(goal_assist[1])
        else:
            temp_goal_list.append(goal_assist[0])
    if op_goal != 0:
        temp_goal_list += ['상대'] * op_goal
        temp_assist_list += ['NULL'] * op_goal
else:
    temp_goal_list, temp_assist_list = ['NULL'], ['NULL']
print(temp_goal_list)
print(temp_assist_list)

['후', '핵', '후', '상대', '상대']
['범', '삐', '삐', 'NULL', 'NULL']


In [208]:
temp = datetime.strptime('2024-09-09 22:00:00', "%Y-%m-%d %H:%M:%S")
print(temp)
time_delta = timedelta(minutes=15)
for i in range(1, 4):
    print(temp)
    temp += time_delta

2024-09-09 22:00:00
2024-09-09 22:00:00
2024-09-09 22:15:00
2024-09-09 22:30:00


In [209]:
type(temp)

datetime.datetime

In [210]:
str(temp)

'2024-09-09 22:45:00'

In [211]:
type(str(temp))

str

In [143]:
# 경기 기록 텍스트 입력
# record_text = '''8/23 금 용산 싼다 골도움
# 참: 핵 삐 트 범 뀨 후 잡 뭉 _8명

# 1Q) 3:2 승
# 후 범
# 핵 삐
# 후 삐

# 2Q) 2:2 무
# 핵 뀨
# 후 잡

# 3Q) 1:0 승
# 뭉 후

# 4Q) 5:2 승
# 삐 뭉
# 삐 잡
# 삐 트
# 뭉 잡
# 잡 삐

# 5Q) 2:2 무
# 핵 삐
# 삐 핵

# 6Q) 1:0 승
# 뭉 후

# 7Q) 1:3 패
# 뭉 삐

# 8Q) 5:0 승
# 잡 핵
# 잡 트
# 잡 핵
# 잡
# 잡 뀨

# 5승 2무 1패'''

In [ ]:
df = pd.DataFrame({'ID':id_list, 'datetime':datetime_list, 'quarter':quarter_list, 'goal':goal_list, 'assist':assist_list, 'opteamname':opteamname_list})
df

,ID,datetime,quarter,goal,assist,opteamname
0,1,2024-09-06 21:00:00,1,잡,트,아일랜드fc
1,2,2024-09-06 21:00:00,1,범,핵,아일랜드fc
2,3,2024-09-06 21:00:00,1,상대,NULL,아일랜드fc
3,4,2024-09-06 21:15:00,2,뭉,후,아일랜드fc
4,5,2024-09-06 21:15:00,2,후,뭉,아일랜드fc
5,6,2024-09-06 21:15:00,2,뭉,NULL,아일랜드fc
6,7,2024-09-06 21:30:00,3,NULL,NULL,아일랜드fc
7,8,2024-09-06 21:45:00,4,뀨,잡,아일랜드fc
8,9,2024-09-06 21:45:00,4,상대,NULL,아일랜드fc
9,10,2024-09-06 22:00:00,5,후,뭉,아일랜드fc


# 경기 기록

## 2024-08-23 경기

In [ ]:
# fc intentar
# 22시

# 8/23 금 용산 싼다 골도움
# 참: 핵 삐 트 범 뀨 후 잡 뭉 _8명

# 1Q) 3:2 승
# 후 범
# 핵 삐
# 후 삐

# 2Q) 2:2 무
# 핵 뀨
# 후 잡

# 3Q) 1:0 승
# 뭉 후

# 4Q) 5:2 승
# 삐 뭉
# 삐 잡
# 삐 트
# 뭉 잡
# 잡 삐

# 5Q) 2:2 무
# 핵 삐
# 삐 핵

# 6Q) 1:0 승
# 뭉 후

# 7Q) 1:3 패
# 뭉 삐

# 8Q) 5:0 승
# 잡 핵
# 잡 트
# 잡 핵
# 잡
# 잡 뀨

# 5승 2무 1패

## 2024-09-06 경기

In [ ]:
# 아일랜드fc
# 21시

# 9/6 금 용산 싼다 골도움
# 참: 핵 트 잡 뭉 범 뀨 쫑 삐 후 _9명

# 1Q) 2:1 승
# 잡 트
# 범 핵

# 2Q) 3:0 승
# 뭉 후
# 후 뭉
# 뭉

# 3Q) 0:0 무

# 4Q) 1:1 무
# 뀨 잡

# 5Q) 1:0 승
# 후 뭉

# 6Q) 1:1 무
# 삐 트

# 7Q) 0:0 무

# 8Q) 3:1 승
# 삐 범
# 삐 뀨
# 뀨 삐

# 4승 4무

# End